In [1]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import pandas as pd
import nltk
import random
from collections import Counter
import os
import pickle as pkl
from sklearn import datasets
from gensim.models import Word2Vec
import os
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from nltk import WordNetLemmatizer, word_tokenize, bigrams, ngrams, RegexpTokenizer
import gensim
from cycler import cycler
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
#cmap=plt.cm.tab10
#c = cycler('color', cmap(np.linspace(0,1,10)))
#plt.rcParams["axes.prop_cycle"] = c
from gensim import corpora, models, similarities, matutils
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec

lemmatize = WordNetLemmatizer()

In [2]:
def clean_text(text):
    tokens = word_tokenize(text)
    clean_tokens = [lemmatize.lemmatize(token.lower().strip()) for token in tokens]
    return ' '.join(clean_tokens)

In [4]:

with open('messages.pkl','rb') as f:
    messages=pkl.load(f)
    
filename = 'text_files/inspiration_quotes.txt'
with open(filename, "r") as file:
    quotes = file.read()
    
quotes_clean = quotes.replace('\n',' ')
tokenizer = RegexpTokenizer(r'[“”]', gaps=True)
quote_clean2 = [quote for quote in tokenizer.tokenize(quotes_clean) if quote[1] != '—']

In [5]:
import string
bad_list=['!','?',':','TANG','INA','PUTANG']
def printify(s):
    printable = set(string.printable)
    return ''.join(list(filter(lambda x: (x in printable) and (x not in bad_list), s)))

import enchant
d = enchant.Dict("en_US")
d.check("Hello")

def anglify(m):
    return ' '.join([x for x in word_tokenize(printify(m)) if d.check(x)])

anglify(messages[4323])

'I do care'

In [6]:
long_messages=[''.join(anglify(m)) for m in messages if len(m.split()) >50]

In [13]:
#list(ngrams(word_tokenize('hi how are you doing'),3))

def make_following(quote_clean2):
    following = {}
    count = {}
    for quote in quote_clean2:
        words = quote.split()
        for index, word in enumerate(words[:-1]):
            # ensure the word is in the dictionary
            if word not in following:
                following[word] = []
                count[word] = 0

            # append the next word
     
            following[word].append(words[index + 1])
            count[word] += 1

        if words:
            last_word = words[-1]
            if last_word not in following:
                following[last_word] = []
                count[last_word] = 0
    return following

def make_bigram(quote_clean2):
    bigrams = {}
    for quote in quote_clean2:
        words = quote.split()
        for w1, w2, w3 in zip(words[:-2], words[1:-1], words[2:]):
            bigram = (w1,w2)
            if bigram not in bigrams:
                bigrams[bigram] = [w3]
            else:
                bigrams[bigram].append(w3)
    return bigrams


def generate_text_bigram(following_dict, bigram_dict, num_words):
    # randomly select a word.
    # Ideas for improvements: 
    # - could mark words as sentence starters
    # - could weight by frequency.
    
    all_bigrams = list(bigram_dict.keys())
    current_bigram = random.choice(all_bigrams)
    text = list(current_bigram)
    followers=list(following_dict.keys())
    
    
    
    for _ in range(num_words - 1):
        if current_bigram in all_bigrams:
            next_word = random.choice(bigram_dict[current_bigram])
        else:
            
            if len(text)==0:
                next_word='of'
            else:

                next_word = random.choice(followers)

        text.append(next_word)
        current_bigram = (current_bigram[1], next_word)

    return ' '.join(text)

def cray_maker(text, num_words):
    # randomly select a word.
    # Ideas for improvements: 
    # - could mark words as sentence starters
    # - could weight by frequency.
    
    bigram_dict=make_bigram(text)
    following_dict=make_following(text)
    
    all_bigrams = list(bigram_dict.keys())
    current_bigram = random.choice(all_bigrams)
    text = list(current_bigram)
    followers=list(following_dict.keys())
    
    
    
    for _ in range(num_words - 1):
        if current_bigram in all_bigrams:
            next_word = random.choice(bigram_dict[current_bigram])
        else:
            
            if len(text)==0:
                next_word='of'
            else:

                next_word = random.choice(followers)

        text.append(next_word)
        current_bigram = (current_bigram[1], next_word)

    return ' '.join(text)

In [14]:
cray_maker(long_messages,20)

'to pages I never even did shit . I your smile 2 . excited Karon next month including me ... .'

In [61]:
'will never be fulfilled. If your happiness depends on money, you'

'will never be fulfilled. If your happiness depends on money, you'

In [ ]:
'life’s big prizes. The Pulitzer. The Nobel. Oscars. The World Cup

In [ ]:
'Life is a song, sing it. Life is a duty, complete'

In [ ]:
'...and women - merely players. They have their exits and their entrances'

Finally, let's generate some examples and save them for later.

In [22]:
markov_data=[]
for _ in range(1000):
    new=[cray_maker(long_messages,10),0]
    markov_data.append(new)

with open('markov_data.pkl','wb') as f:
    pkl.dump(markov_data,f)